# Cleaning CONTACTO dataset

Imports:

In [1]:
import os

import pandas as pd
import numpy as np

from dotenv import load_dotenv

from core_ds4a_project import cleaning, columns as project_columns, datasets

%load_ext autoreload
%autoreload 1
%aimport core_ds4a_project, core_ds4a_project.cleaning, core_ds4a_project.columns, core_ds4a_project.datasets

In [2]:
pd.set_option("display.max_columns", None)

Environment variables:

In [3]:
load_dotenv('envvars')

ROOT_DATA_PATH = os.environ.get('ROOT_DATA_PATH')
RAW_DATA_PATH = os.environ.get('RAW_DATA_PATH') or f'{ROOT_DATA_PATH}/raw'

Reading data:

In [4]:
df = (
    datasets.read_contacto(RAW_DATA_PATH, raw=True)
)

df.shape

(60043, 70)

## Schema of columns for raw data

In [5]:
df[df.columns.sort_values()].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60043 entries, 0 to 60042
Data columns (total 70 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   	
Entidad Financiera                                             75 non-null     object 
 1   	
Lugar de Expedición                                            34290 non-null  object 
 2   	
Mujer Cabeza de Familia                                        60043 non-null  object 
 3   Actividad CIIU Primaria                                          25602 non-null  object 
 4   Actividad Económica                                              52094 non-null  object 
 5   Administra Recursos Públicos                                     59976 non-null  object 
 6   Asignado a                                                       60043 non-null  object 
 7   Autoriza el envió de información por med

## Renaming columns

In [6]:
df.columns = cleaning.normalize_columns_name(df.columns)

df.columns.sort_values()  # sorting for presentational purposes only

Index(['ACTIVIDAD_CIIU_PRIMARIA', 'ACTIVIDAD_ECONOMICA',
       'ADMINISTRA_RECURSOS_PUBLICOS', 'ASIGNADO_A',
       'AUTORIZA_EL_ENVIO_DE_INFORMACION_POR_MEDIO_DE_CORREO_ELCTRONICO',
       'AUTORIZA_LA_CONSULTA_EN_LAS_CENTRALES_DE_RIESGO',
       'BANCO_MONEDA_EXTRANJERA', 'BARRIO_COMERCIAL', 'BARRIO_RESIDENCIA',
       'CENTRO_DE_COSTO', 'CIUDAD_COMERCIAL', 'CIUDAD_EMPRESA',
       'CIUDAD_RESIDENCIA', 'CLIENTE', 'CUENTA_MONEDA_EXTRANJERA',
       'DECLARA_RENTA', 'DESCRIPCION_DE_LAS_OPERACIONES_EN_MONEDA_EXTRANJERA',
       'DESCRIPCION_OTROS_INGRESOS', 'DIRECCION_COMERCIAL',
       'DIRECCION_DE_RESIDENCIA', 'DIRECCION_EMPRESA', 'DIR_CORRESPONDENCIA',
       'EGRESOS_MENSUALES', 'EMAIL', 'EMPRESA_DONDE_TRABAJA',
       'ENTIDAD_FINANCIERA', 'ESTADO_CIVIL', 'FAX', 'FAX_EMPRESA',
       'FECHA_DE_EXPEDICION_CEDULA', 'FECHA_NACIMIENTO', 'GENERO',
       'INGRESO_MENSUAL', 'JORNADA_LABORAL', 'LUGAR_DE_EXPEDICION',
       'MEDIO_DE_PAGO', 'MONEDA', 'MOVIL', 'MUJER_CABEZA', 'NATURALEZA'

## Column: CLIENTE

In [7]:
df.shape[0], df['CLIENTE'].nunique()

(60043, 60043)

In [8]:
# Used to update some categorical variables
RE_NOMEN_PATT = r'\s\|.*$'
RE_CODE_PATT = r'^.*\s\|'

## Relevant columns

Notes:
- MUJER_CABEZA and RESPONSABLE_HOGAR are not parsed with regular expressions as those columns are unique across datasets.

### ACTIVIDAD_ECONOMICA, ACTIVIDAD_CIIU_PRIMARIA

In [9]:
df[['ACTIVIDAD_ECONOMICA', 'ACTIVIDAD_CIIU_PRIMARIA']].describe()

,ACTIVIDAD_ECONOMICA,ACTIVIDAD_CIIU_PRIMARIA
count,52094,25602
unique,10,296
top,Negocio o Microempresa |NM,CRÍA DE GANADO BOVINO Y BUFALINO |141|A
freq,44918,3298


In [10]:
df['ACTIVIDAD_ECONOMICA'].value_counts(dropna=False)

Negocio o Microempresa |NM       44918
NaN                               7949
Empleado |EM                      3886
Hogar |HO                         1612
Desconocida |NN                    944
Profesional Independiente |PI      334
Pensionado |PE                     147
Depende economicamente |DO         122
Cesante |CE                         47
Rentas de Capital |RE               43
Estudiante |ES                      41
Name: ACTIVIDAD_ECONOMICA, dtype: int64

In [11]:
df['ACTIVIDAD_CIIU_PRIMARIA'].value_counts(dropna=False)

NaN                                                                                                                                                                34441
CRÍA DE GANADO BOVINO Y BUFALINO |141|A                                                                                                                             3298
COMERCIO AL POR MENOR EN ESTABLECIMIENTOS NO ESPECIALIZADOS CON SURTIDO COMPUESTO PRINCIPALMENTE POR ALIMENTOS BEBIDAS O TABACO |4711|G                             2119
OTROS CULTIVOS PERMANENTES N.C.P. |129|A                                                                                                                            1974
ACTIVIDADES INMOBILIARIAS REALIZADAS CON BIENES PROPIOS O ARRENDADOS |6810|L                                                                                        1381
                                                                                                                                                           

### ESTADO_CIVIL

In [12]:
df['ESTADO_CIVIL'].value_counts(dropna=False)

Union libre |U    19162
NaN               12823
Casado |C         12661
Soltero |S        12189
Divorciado |D      1921
Viudo |V           1235
Otro |O              52
Name: ESTADO_CIVIL, dtype: int64

In [13]:
# All records match format "NOMENCLATURE |CODE"
assert df['ESTADO_CIVIL'].str.match(r'.*\s\|\w', ).all(), "Not all records match expected pattern"

In [14]:
df['ESTADO_CIVIL_COD'] = (
    df['ESTADO_CIVIL']
    .str.replace(r'.*\s\|', '', regex=True)
    .astype('category')
)
df['ESTADO_CIVIL'] = (
    df['ESTADO_CIVIL']
    .str.replace(r'\s\|.*', '', regex=True)
    .astype('category')
)

df[['ESTADO_CIVIL_COD', 'ESTADO_CIVIL']].value_counts(dropna=False)

ESTADO_CIVIL_COD  ESTADO_CIVIL
U                 Union libre     19162
C                 Casado          12661
S                 Soltero         12189
D                 Divorciado       1921
V                 Viudo            1235
O                 Otro               52
dtype: int64

In [15]:
df[['ESTADO_CIVIL_COD', 'ESTADO_CIVIL']].isna().sum()

ESTADO_CIVIL_COD    12823
ESTADO_CIVIL        12823
dtype: int64

### FECHA_NACIMIENTO

There are some anomalies presummably associated with typos that are fixed at following:

In [16]:
re_pattern_date_year = r'(?:\d{1,2}/\d{1,2}/)(?P<year>\d{4})'
year = df['FECHA_NACIMIENTO'].str.extract(re_pattern_date_year)['year']
ind = year.astype(float) < 1900

df.loc[ind, 'FECHA_NACIMIENTO']

39766    25/08/0977
39869    08/09/1798
43244    20/05/1071
46272    23/10/1057
47789    21/06/1772
48906    28/06/1885
54378    19/01/1074
58154    06/09/0965
59764    27/10/1058
59963    01/03/1194
Name: FECHA_NACIMIENTO, dtype: object

In [17]:
fix_typos_dict = {
    '25/08/0977': '25/08/1977',
    '08/09/1798': '08/09/1978',
    '20/05/1071': '20/05/1971',
    '23/10/1057': '23/10/1957',
    '21/06/1772': '21/06/1972',
    '28/06/1885': '28/06/1985',
    '19/01/1074': '19/01/1974',
    '06/09/0965': '06/09/1965',
    '27/10/1058': '27/10/1958',
    '01/03/1194': '01/03/1994',
}

df['FECHA_NACIMIENTO'] = pd.to_datetime(
    df['FECHA_NACIMIENTO'].replace(fix_typos_dict),
    format='%d/%m/%Y'
)

df['FECHA_NACIMIENTO']


0       1955-03-30
1       1977-11-01
2       1980-10-24
3       1986-02-03
4       1984-09-21
           ...    
60038   1995-12-08
60039   1990-03-09
60040   1998-12-20
60041   1958-12-18
60042   1992-05-02
Name: FECHA_NACIMIENTO, Length: 60043, dtype: datetime64[ns]

### GENERO

In [18]:
df['GENERO'].value_counts(dropna=False)

Femenino |F     36505
Masculino |M    23501
NaN                37
Name: GENERO, dtype: int64

In [19]:
# All records match format "NOMENCLATURE |CODE"
assert df['GENERO'].str.match(r'.*\s\|\w', ).all(), "Not all records match expected pattern"

In [20]:
df['GENERO_COD'] = (
    df['GENERO']
    .str.replace(r'.*\s\|', '', regex=True)
    .astype('category')
)
df['GENERO'] = (
    df['GENERO']
    .str.replace(r'\s\|.*', '', regex=True)
    .astype('category')
)

df[['GENERO_COD', 'GENERO']].value_counts(dropna=False)

GENERO_COD  GENERO   
F           Femenino     36505
M           Masculino    23501
dtype: int64

### MUJER_CABEZA

In [21]:
df['MUJER_CABEZA'].value_counts(dropna=False)

No |N    55890
Si |Y     4153
Name: MUJER_CABEZA, dtype: int64

In [22]:
df['MUJER_CABEZA'] = df['MUJER_CABEZA'].astype('category')

df['MUJER_CABEZA'].value_counts()

No |N    55890
Si |Y     4153
Name: MUJER_CABEZA, dtype: int64

### NIVEL_ESTUDIOS

In [23]:
df['NIVEL_ESTUDIOS'].value_counts(dropna=False)

Secundaria |S         23452
Primaria |P           18927
NaN                    7170
Tecnica |T             4647
Universitaria |U       3554
Tecnológica |V         1437
Analfabetismo |A        296
Especializacion |E      242
No indica |X            171
No escolarizado |C      107
Magister |M              19
Maestria |I              18
Doctorado |D              3
Name: NIVEL_ESTUDIOS, dtype: int64

In [24]:
# All records match format "NOMENCLATURE |CODE"
assert df['NIVEL_ESTUDIOS'].str.match(r'^.*\s\|\w$', ).all(), "Not all records match expected pattern"

In [25]:
df['NIVEL_ESTUDIOS_COD'] = (
    df['NIVEL_ESTUDIOS']
    .str.replace(r'.*\s\|', '', regex=True)
    .astype('category')
)
df['NIVEL_ESTUDIOS'] = (
    df['NIVEL_ESTUDIOS']
    .str.replace(r'\s\|.*', '', regex=True)
    .astype('category')
)

df[['NIVEL_ESTUDIOS_COD', 'NIVEL_ESTUDIOS']].value_counts(dropna=False)


NIVEL_ESTUDIOS_COD  NIVEL_ESTUDIOS 
S                   Secundaria         23452
P                   Primaria           18927
T                   Tecnica             4647
U                   Universitaria       3554
V                   Tecnológica         1437
A                   Analfabetismo        296
E                   Especializacion      242
X                   No indica            171
C                   No escolarizado      107
M                   Magister              19
I                   Maestria              18
D                   Doctorado              3
dtype: int64

### OCUPACION, OFICIO

In [26]:
df['OCUPACION'].value_counts(dropna=False)

Independiente |2    43013
NaN                 11511
Empleado |1          3041
No Aplica |0         1407
Hogar |5              887
Pensionado |3         114
Estudiante |4          43
Cesante |6             27
Name: OCUPACION, dtype: int64

In [27]:
df['OFICIO'].value_counts(dropna=False)

NaN                               19333
GANADERIA                          3762
AGRICULTOR                         2287
TIENDA                             1488
ARRENDADOR                         1162
                                  ...  
 Comercio al por menor de bebi        1
ELABORACION DE CHINCHORROS            1
9529 Mantenimiento y reparació        1
VENTA DE CATALOGOS Y ARRIENDO         1
LATONERO Y PINTOR AUTOMOTRIZ          1
Name: OFICIO, Length: 6765, dtype: int64

In [28]:
df[['OCUPACION', 'OFICIO']].nunique()

OCUPACION       7
OFICIO       6764
dtype: int64

### PROFESION

In [29]:
df['PROFESION'].value_counts(dropna=False)

NaN                          33688
DESCONOCIDA |D               12412
AGRICULTURA |204              3645
GANADERIA |205                3043
ADMON DE NEGOCIOS |107        1632
                             ...  
QUIMICA FARMACEUTICA |186        1
ARQUEOLOGIA |123                 1
TECN. QUIMICA |191               1
ANTROPOLOGIA |122                1
INGENIERIA MINAS |170            1
Name: PROFESION, Length: 101, dtype: int64

In [30]:
# All records match format "NOMENCLATURE |CODE"
assert df['PROFESION'].str.match(r'.*\s\|\w', ).all(), "Not all records match expected pattern"

In [31]:
df['PROFESION_COD'] = (
    df['PROFESION']
    .str.replace(r'.*\s\|', '', regex=True)
    .astype('category')
)
df['PROFESION'] = (
    df['PROFESION']
    .str.replace(r'\s\|.*', '', regex=True)
    .astype('category')
)

df[['PROFESION_COD', 'PROFESION']].value_counts(dropna=False)

PROFESION_COD  PROFESION              
D              DESCONOCIDA                12412
204            AGRICULTURA                 3645
205            GANADERIA                   3043
107            ADMON DE NEGOCIOS           1632
106            ADMON DE EMPRESAS           1278
                                          ...  
186            QUIMICA FARMACEUTICA           1
178            OCEANOGRAFIA                   1
175            MICROBIOLOGIA                  1
170            INGENIERIA MINAS               1
157            ING. CATASTRAL/GEODESIA        1
Length: 100, dtype: int64

### RESPONSABLE_DE_HOGAR

In [32]:
df['RESPONSABLE_DE_HOGAR'].value_counts(dropna=False)

NaN      26540
Si |Y    17996
No |N    15507
Name: RESPONSABLE_DE_HOGAR, dtype: int64

In [33]:
df['RESPONSABLE_DE_HOGAR'] = df['RESPONSABLE_DE_HOGAR'].astype('category')

df['RESPONSABLE_DE_HOGAR'].value_counts()

Si |Y    17996
No |N    15507
Name: RESPONSABLE_DE_HOGAR, dtype: int64

### SUELDO_BASICO

Question: zero means there is not salary? or unknown?

In [34]:
df['SUELDO_BASICO'].value_counts(dropna=False)

0.0          45506
NaN           7153
1200000.0      406
2000000.0      380
1500000.0      352
             ...  
1327891.0        1
2092000.0        1
899441.0         1
2367000.0        1
942000.0         1
Name: SUELDO_BASICO, Length: 1656, dtype: int64

### TIPO_DE_CLIENTE

In [35]:
df['TIPO_DE_CLIENTE'].value_counts(dropna=False)

Microfinanciero |1           50048
Codeudor no cliente |4        8609
Gestion social |2              742
Mixto |3                       379
Empleado Req.Servicios |9      149
Proveedor |7                   110
Fondeador |5                     6
Name: TIPO_DE_CLIENTE, dtype: int64

In [36]:
df['TIPO_DE_CLIENTE'] = (
    df['TIPO_DE_CLIENTE']
    .str.replace(r'\s\|.*', '', regex=True)
    .astype('category')
)

df['TIPO_DE_CLIENTE'].value_counts(dropna=False)

Microfinanciero           50048
Codeudor no cliente        8609
Gestion social              742
Mixto                       379
Empleado Req.Servicios      149
Proveedor                   110
Fondeador                     6
Name: TIPO_DE_CLIENTE, dtype: int64

### TIPO_DE_IDENTIFICACION

Question: negocio?

In [37]:
# NIT!
df['TIPO_DE_IDENTIFICACION'].value_counts(dropna=False)

Cedula |C                   59888
Cedula de Extranjería |E       68
Nit |A                         57
Tarjeta de Identidad |T        25
Registro Civil |R               2
Carnet Diplomático |D           2
Otros |O                        1
Name: TIPO_DE_IDENTIFICACION, dtype: int64

In [38]:
ind = df['TIPO_DE_IDENTIFICACION'] == 'Nit |A'

ind.sum()

57

In [39]:
df[ind].drop(columns=project_columns.CONTACTO_DISCARDED_COLUMNS)

,CLIENTE,TIPO_UBICACION,SUCURSAL,TIPO_DE_CLIENTE,SUELDO_BASICO,ACTIVIDAD_ECONOMICA,TIPO_VIVIENDA,TIPO_DE_IDENTIFICACION,OFICIO,ESTADO_CIVIL,GENERO,FECHA_NACIMIENTO,NIVEL_ESTUDIOS,PROFESION,MUJER_CABEZA,RESPONSABLE_DE_HOGAR,OCUPACION,ACTIVIDAD_CIIU_PRIMARIA,ESTADO_CIVIL_COD,GENERO_COD,NIVEL_ESTUDIOS_COD,PROFESION_COD
6543,FA6544,URBANA |1,VILLANUEVA |11,Microfinanciero,0.0,NaN,Propia |P,Nit |A,NaN,Union libre,Femenino,1981-08-18,Secundaria,NaN,No |N,NaN,NaN,NaN,U,F,S,NaN
17829,FA17830,URBANA |1,AGUAZUL |4,Microfinanciero,0.0,Negocio o Microempresa |NM,Propia |P,Nit |A,panaderia,Union libre,Femenino,1971-09-18,Secundaria,DESCONOCIDA,No |N,No |N,Independiente |2,NaN,U,F,S,D
18241,FA18242,URBANA |1,YOPAL |1,Microfinanciero,0.0,Negocio o Microempresa |NM,Familiar |F,Nit |A,NaN,Union libre,Femenino,1987-07-05,Secundaria,DESCONOCIDA,No |N,No |N,Independiente |2,EXPENDIO A LA MESA DE COMIDAS PREPARADAS |5611|I,U,F,S,D
22688,FA22689,URBANA |1,GRANADA |6,Microfinanciero,0.0,Negocio o Microempresa |NM,Propia |P,Nit |A,DANYCELL COMUNICACIONES,Union libre,Femenino,1979-08-27,Tecnica,DESCONOCIDA,No |N,No |N,Independiente |2,NaN,U,F,T,D
23894,FA23895,URBANA |1,YOPAL |1,Empleado Req.Servicios,0.0,Profesional Independiente |PI,Propia |P,Nit |A,CONTADURIA,Soltero,Masculino,1973-06-09,Universitaria,CONTADURIA,No |N,No |N,Independiente |2,INVESTIGACIONES Y DESARROLLO EXPERIMENTAL EN E...,S,M,U,134
34384,FA34385,URBANA |1,YOPAL |1,Microfinanciero,0.0,Negocio o Microempresa |NM,Propia |P,Nit |A,HOTEL,Divorciado,Femenino,1960-10-16,Secundaria,NaN,No |N,NaN,Independiente |2,ACTIVIDADES INMOBILIARIAS REALIZADAS A CAMBIO ...,D,F,S,NaN
35499,FA35500,URBANA |1,YOPAL |1,Microfinanciero,0.0,Negocio o Microempresa |NM,Propia |P,Nit |A,TELECOMUNICACIONES,Casado,Masculino,1974-04-11,Tecnica,NaN,No |N,NaN,Independiente |2,MANTENIMIENTO Y REPARACIÓN DE EQUIPOS DE COMUN...,C,M,T,NaN
35892,FA35893,RURAL |2,AGUAZUL |4,Gestion social,0.0,Desconocida |NN,Propia |P,Nit |A,AGROPECUARIO,Union libre,Masculino,1977-08-13,Universitaria,NaN,No |N,NaN,No Aplica |0,INVESTIGACIONES Y DESARROLLO EXPERIMENTAL EN E...,U,M,U,NaN
37182,FA37183,URBANA |1,YOPAL |1,Microfinanciero,NaN,NaN,NaN,Nit |A,NaN,NaN,Masculino,NaT,NaN,NaN,No |N,NaN,NaN,MANTENIMIENTO Y REPARACIÓN DE APARATOS ELECTRÓ...,NaN,M,NaN,NaN
37311,FA37312,URBANA |1,YOPAL |1,Gestion social,0.0,Hogar |HO,Propia |P,Nit |A,COMERCIANTE,Union libre,Femenino,1978-02-21,Secundaria,NaN,No |N,NaN,Hogar |5,INVESTIGACIONES Y DESARROLLO EXPERIMENTAL EN E...,U,F,S,NaN


### TIPO_UBICACION

In [40]:
df['TIPO_UBICACION'].value_counts(dropna=False)

URBANA |1    41663
RURAL |2     18380
Name: TIPO_UBICACION, dtype: int64

In [41]:
# All records match format "NOMENCLATURE |CODE"
assert df['TIPO_UBICACION'].str.match(r'.*\s\|\w', ).all(), "Not all records match expected pattern"

In [42]:
df['TIPO_UBICACION_COD'] = (
    df['TIPO_UBICACION']
    .str.replace(r'.*\s\|', '', regex=True)
    .astype('category')
)
df['TIPO_UBICACION'] = (
    df['TIPO_UBICACION']
    .str.replace(r'\s\|.*', '', regex=True)
    .astype('category')
)

df[['TIPO_UBICACION_COD', 'TIPO_UBICACION']].value_counts(dropna=False)


TIPO_UBICACION_COD  TIPO_UBICACION
1                   URBANA            41663
2                   RURAL             18380
dtype: int64

### TIPO_DE_VIVIENDA

In [43]:
df['TIPO_VIVIENDA'].value_counts(dropna=False)

Propia |P                   22942
NaN                         13729
Otra |O                      9356
Familiar |F                  8487
Arriendo |A                  5515
Inmueble con Hipoteca |H       14
Name: TIPO_VIVIENDA, dtype: int64

In [44]:
# All records match format "NOMENCLATURE |CODE"
assert df['TIPO_VIVIENDA'].str.match(r'.*\s\|\w', ).all(), "Not all records match expected pattern"

In [45]:
df['TIPO_VIVIENDA_COD'] = (
    df['TIPO_VIVIENDA']
    .str.replace(r'.*\s\|', '', regex=True)
    .astype('category')
)
df['TIPO_VIVIENDA'] = cleaning.clean_tipo_vivienda(
    df['TIPO_VIVIENDA'].str.replace(r'\s\|.*', '', regex=True)
)

df[['TIPO_VIVIENDA_COD', 'TIPO_VIVIENDA']].value_counts(dropna=False)


TIPO_VIVIENDA_COD  TIPO_VIVIENDA
P                  PROPIA           22942
O                  OTRA              9356
F                  FAMILIAR          8487
A                  ARRIENDO          5515
H                  HIPOTECA            14
dtype: int64

## Discarded columns

The criteria to discard columns is:
- More than 90% of values are NaN.
- Columns describe a single behavior and/or characteristics.
- Content is not meaningful nor useful for analysis such as communication-related columns.

The following are columns which more than 90% of values are NaN:

In [46]:
limit = round(df.shape[0] * 0.9)
ind_count_na = (df.isna().sum() > limit)

df.columns[ind_count_na].sort_values().values

array(['BANCO_MONEDA_EXTRANJERA', 'CIUDAD_EMPRESA',
       'CUENTA_MONEDA_EXTRANJERA',
       'DESCRIPCION_DE_LAS_OPERACIONES_EN_MONEDA_EXTRANJERA',
       'DESCRIPCION_OTROS_INGRESOS', 'DIRECCION_EMPRESA',
       'EGRESOS_MENSUALES', 'ENTIDAD_FINANCIERA', 'FAX', 'FAX_EMPRESA',
       'INGRESO_MENSUAL', 'MONEDA',
       'NOMBRE_DEL_CARGO_QUE_DESEMPENA_EN_LA_EMPRESA', 'OTROS_INGRESOS',
       'PAIS_MONEDA_EXTRANJERA', 'TELEFONO_EMPRESA', 'TIPO_DE_CONTRATO',
       'VALOR_OTROS_DESCUENTOS_DE_NOMINA', 'VENCIMIENTO_DEL_CONTRATO'],
      dtype=object)

In [47]:
(df.columns[ind_count_na].isin(project_columns.CONTACTO_DISCARDED_COLUMNS)).all()

True

### DECLARA_RENTA

In [48]:
df['DECLARA_RENTA'].value_counts(dropna=False)

No |N    59970
NaN         67
Si |Y        6
Name: DECLARA_RENTA, dtype: int64

### EMPRESA_DONDE_TRABAJA

In [49]:
df['EMPRESA_DONDE_TRABAJA'].value_counts(dropna=False).head()

NaN                   36521
GANADERIA               164
ARRENDAMIENTOS           90
ARRIENDOS                77
VENTA POR CATALOGO       61
Name: EMPRESA_DONDE_TRABAJA, dtype: int64

### EGRESOS_MENSUALES

EGRESOS_MENSUALES might be useful but it is discarded as most values are NaN:

In [50]:
df['EGRESOS_MENSUALES'].value_counts(dropna=False)

NaN          58946
500000.0        23
350000.0        19
400000.0        16
0.0             14
             ...  
1397000.0        1
3499000.0        1
2193000.0        1
2293000.0        1
2294000.0        1
Name: EGRESOS_MENSUALES, Length: 839, dtype: int64

### INGRESO_MENSUAL

INGRESO_MENSUAL might be useful but it is discarded as most values are NaN:

In [51]:
df['INGRESO_MENSUAL'].value_counts(dropna=False)

NaN           58950
2000000.0        45
3000000.0        44
1500000.0        37
1200000.0        36
              ...  
1890000.0         1
3333500.0         1
15950000.0        1
10150000.0        1
4320000.0         1
Name: INGRESO_MENSUAL, Length: 449, dtype: int64

### NATURALEZA

In [52]:
df['NATURALEZA'].value_counts(dropna=False)

Natural |N    60043
Name: NATURALEZA, dtype: int64

### NOMBRE_DEL_CARGO_QUE_DESEMPENA_EN_LA_EMPRESA

In [53]:
df['NOMBRE_DEL_CARGO_QUE_DESEMPENA_EN_LA_EMPRESA'].value_counts(dropna=False)

NaN                58663
Default             1305
MICROEMPRESARIO       57
UNICO                 18
Name: NOMBRE_DEL_CARGO_QUE_DESEMPENA_EN_LA_EMPRESA, dtype: int64

### POBLACION_VULNERABLE

In [54]:
df['POBLACION_VULNERABLE'].value_counts(dropna=False)

No Aplica |NA                             31505
NaN                                       28330
Madre cabeza de Familia |MC                 126
Joven |JO                                    56
Desplazado |DE                                9
Reinsertado |RE                               7
Discapacitado |DI                             5
Palenqueros |PA                               2
Comunidad Indígena |IN                        2
Afrodescendientes negros o mulatos |AF        1
Name: POBLACION_VULNERABLE, dtype: int64

### REGIMEN_DE_IMPUESTOS

In [55]:
df['REGIMEN_DE_IMPUESTOS'].value_counts(dropna=False)

Régimen Simplificado |S    60002
Régimen comun |C              41
Name: REGIMEN_DE_IMPUESTOS, dtype: int64

### REGIMEN_DE_IVA

In [56]:
df['REGIMEN_DE_IVA'].value_counts(dropna=False)

Régimen Simplificado |S    60004
Régimen Común |C              39
Name: REGIMEN_DE_IVA, dtype: int64

### REGIMEN_DE_RENTA

In [57]:
df['REGIMEN_DE_RENTA'].value_counts(dropna=False)

Contribuyente |T       60040
Empleados |E               2
No Contribuyente |N        1
Name: REGIMEN_DE_RENTA, dtype: int64

### REALIZA_OPERACIONES_EN_MONEDA_EXTRANJERA

REALIZA_OPERACIONES_EN_MONEDA_EXTRANJERA is irrelevant as all values are "No |N" but single one:

In [58]:
df['REALIZA_OPERACIONES_EN_MONEDA_EXTRANJERA'].value_counts(dropna=False)

No |N    59975
NaN         67
Si |Y        1
Name: REALIZA_OPERACIONES_EN_MONEDA_EXTRANJERA, dtype: int64

### TIPO_DE_CONTRATO

TIPO_DE_CONTRATO might be useful but it is discarded as most values are NaN:

In [59]:
df['TIPO_DE_CONTRATO'].value_counts(dropna=False)

NaN                                 58280
Término Indefinido |TI               1325
Término Fijo |TF                      138
Otras Formas de Contratación |OC      125
Culminación de Obra |CO               103
Prestación de Servicios |PS            58
Planta Temporal |PT                     4
Libre Nombramiento |LN                  4
Carrera Administrativa |CA              4
Provisionalidad |PR                     2
Name: TIPO_DE_CONTRATO, dtype: int64

### TIPO_DE_CONTRIBUYENTE

In [60]:
df['TIPO_DE_CONTRIBUYENTE'].value_counts(dropna=False)

Ordinario |D    60043
Name: TIPO_DE_CONTRIBUYENTE, dtype: int64

### UBICACION_2

UBICACION_2 is discarded considering all location-related columns are not being considered.

In [61]:
df['UBICACION_2'].value_counts(dropna=False)

YOPAL |10             9135
VILLAVICENCIO |10     7754
SIN UBICACION |0      6521
ACACIAS |10           3401
PAZ DE ARIPORO |13    3241
                      ... 
TUTAZA |18               2
MESETAS |24              1
CHIQUIZA |19             1
CUCAITA |22              1
TUTA |16                 1
Name: UBICACION_2, Length: 80, dtype: int64

## Schema of columns for clean data

In [62]:
df = df.drop(columns=project_columns.CONTACTO_DISCARDED_COLUMNS)

df[df.columns.sort_values()].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60043 entries, 0 to 60042
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ACTIVIDAD_CIIU_PRIMARIA  25602 non-null  object        
 1   ACTIVIDAD_ECONOMICA      52094 non-null  object        
 2   CLIENTE                  60043 non-null  object        
 3   ESTADO_CIVIL             47220 non-null  category      
 4   ESTADO_CIVIL_COD         47220 non-null  category      
 5   FECHA_NACIMIENTO         53253 non-null  datetime64[ns]
 6   GENERO                   60006 non-null  category      
 7   GENERO_COD               60006 non-null  category      
 8   MUJER_CABEZA             60043 non-null  category      
 9   NIVEL_ESTUDIOS           52873 non-null  category      
 10  NIVEL_ESTUDIOS_COD       52873 non-null  category      
 11  OCUPACION                48532 non-null  object        
 12  OFICIO                   40710 n

## Validating cleaning code

In [63]:
df2 = datasets.read_contacto(dir_path=RAW_DATA_PATH, clean=True)

In [64]:
sorted_cols = df.columns.sort_values()

assert sorted_cols.equals(df2.columns.sort_values()), "Columns do not match"

In [65]:
assert df[sorted_cols].equals(df2[sorted_cols]), "Dataframes are different"

Differences in shapes (if any):

In [66]:
df.shape, df2.shape

((60043, 24), (60043, 24))

In [67]:
ind = pd.Series([df[col].equals(df2[col]) for col in df.columns])

df.columns[~ind]

Index([], dtype='object')